## Goals/approach/data used

The specific datasets used here are posted here.

https://docs.google.com/document/d/1Kos4MTmkEcEqaQUhtKWek8bN3aTNBr4WcDZptRvZOI8/edit 

I'm starting off with manually cleaning them for now and observing recurring issues that I can generalize/automate for. Some of these issues include:
* imputation (when it makes sense, when it doesn't, what kind of imputation)
* information formatting for datetime, location, etc
* data that is split across columns but should be 'one' field
* etc

every dataset we have is for the most part well formed, so i haven't actually needed to test for those issues haha. 

In [83]:
import pandas as pd
import usaddress

In [71]:
df = pd.read_csv('./input/Behavioral_Risk_Factor_Data__Heart_Disease___Stroke_Prevention.csv')

In [5]:
df.columns.values

array(['Year', 'LocationAbbr', 'LocationDesc', 'Datasource',
       'PriorityArea1', 'PriorityArea2', 'PriorityArea3', 'PriorityArea4',
       'Category', 'Topic', 'Indicator', 'Data_Value_Type',
       'Data_Value_Unit', 'Data_Value', 'Data_Value_Alt',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Confidence_Limit_Low', 'Confidence_Limit_High',
       'Break_Out_Category', 'Break_out', 'CategoryID', 'TopicID',
       'IndicatorID', 'Data_Value_TypeID', 'BreakoutCategoryID',
       'BreakOutID', 'LocationID', 'GeoLocation'], dtype=object)

In [21]:
df['Topic'].unique()

array(['Major Cardiovascular Disease', 'Stroke', 'Diabetes',
       'Acute Myocardial Infarction (Heart Attack)',
       'Coronary Heart Disease', 'Cholesterol Abnormalities', 'Nutrition',
       'Obesity', 'Smoking', 'Physical Inactivity', 'Hypertension'],
      dtype=object)

These 'topics' are exactly what we want for metadata tagging; the issue is that most of this could not have been inferred from the headers alone or from the dataset description on the site. 

To quickly identify columns with data that is relevant for metadata tagging, I think we could build a dictionary of medical related terms (pretty trivial) and cross reference subsets of each column with this dictionary. If we find some 'good' columns, we can label and consider those for the tagging stage. If not, it's no big deal - not much overhead.

In [28]:
df['Indicator'].unique()

array(['Prevalence of major cardiovascular disease among US adults (18+); BRFSS',
       'Prevalence of stroke among US adults (18+); BRFSS',
       'Prevalence of diabetes among US adults (18+); BRFSS',
       'Prevalence of acute myocardial infarction (heart attack) among US adults (18+); BRFSS',
       'Prevalence of coronary heart disease among US adults (18+); BRFSS',
       'Prevalence of post-hospitalization rehabilitation among heart attack patients, US adults (18+); BRFSS',
       'Prevalence of cholesterol screening in the past 5 years among US adults (20+); BRFSS',
       'Prevalence of high total cholesterol among US adults (20+); BRFSS',
       'Prevalence of consuming fruits and vegetables less than 5 times per day among US adults (18+); BRFSS',
       'Prevalence of obesity among US adults (20+); BRFSS',
       'Prevalence of healthy weight among US adults (20+); BRFSS',
       'Prevalence of current smoking among US adults (18+); BRFSS',
       'Prevalence of physical i

The only way to tell what the Data Values correspond to is to look at the corresponding 'indicator' value rather than the column header. This is kind of dumb and I can hope that future csvs don't have this feature. We might want to 'unpack' the datasets (e.g. each 'indicator' should be its own datafile and represent a header rather than a column entry) so that each of these indicators are independent datasets/tagged independently (e.g. data about 'consuming fruits and vegetables...' would be a SEPARATE dataset tagged with food+heart disease, data about 'physical inactivity' would be a separate dataset tagged for fitness, etc. it wouldn't make sense to tag all these separate things the same, would it?)

### Part 1: Detecting what constitutes invalid 'null' data

From this/the other datasets posted, there honestly doesn't seem like a huge issue with just straight pandas->read_csv->dropna(subset) based on a manual selection of a subset of columns to pay attention to. That's because all the datasets are already well-formed though.

For this dataframe, it appears just testing for missing entries is totally fine. The question is what columns to test this on; testing on all columns returns more 'missing' data than need be. For example if we dropped rows based on NaN in the 'Data Value Footnote' field for example we'd drop way too much. How to intelligently tell what columns to base this decision on? It appears that we need to manually identify columns that 'matter' first (e.g the footnote doesn't matter).

In [53]:
[[((~df['Data_Value'].isnull()) & (df[col].isnull())).sum(), col] for col in df]

[[0, 'Year'],
 [0, 'LocationAbbr'],
 [0, 'LocationDesc'],
 [0, 'Datasource'],
 [0, 'PriorityArea1'],
 [0, 'PriorityArea2'],
 [0, 'PriorityArea3'],
 [0, 'PriorityArea4'],
 [0, 'Category'],
 [0, 'Topic'],
 [0, 'Indicator'],
 [0, 'Data_Value_Type'],
 [0, 'Data_Value_Unit'],
 [0, 'Data_Value'],
 [0, 'Data_Value_Alt'],
 [57032, 'Data_Value_Footnote_Symbol'],
 [57032, 'Data_Value_Footnote'],
 [1399, 'Confidence_Limit_Low'],
 [1399, 'Confidence_Limit_High'],
 [0, 'Break_Out_Category'],
 [0, 'Break_out'],
 [0, 'CategoryID'],
 [0, 'TopicID'],
 [0, 'IndicatorID'],
 [0, 'Data_Value_TypeID'],
 [0, 'BreakoutCategoryID'],
 [0, 'BreakOutID'],
 [0, 'LocationID'],
 [1399, 'GeoLocation']]

Another point that comes up; this dataset is fucking stupid and has 1399 rows that are already aggregates of the data (the medians the data, but just re-entered as rows, and denoted in the 'Location' field). This could mess up analysis and should be cleaned. Hopefully the data we get isn't that dumb; how to systematically test that? I'll leave it alone for now.

In [63]:
df['LocationDesc'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Median of all states',
       'Washington, DC', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

### Part 2 - Data Imputation 

Is this the data scientists' problem, or our problem? Different imputation methods introduce different biases/is dependent on the nature of the data itself. Do we want to test for whether data is missing approx at random (and just dropping it won't matter much) or what? idk how this is handled typically in ml/data sci

### Part 3 - Manually shaping the dataset

In [87]:
#Takes a mapping of desired column names to ordered columns in the dataset and joins that data together.
#Used to combine data split across different columns (e.g. in the in the 'crimelab' dataset.)

In [ ]:
#Identifies column data that would be useful for classification purposes based on a corpus of medical text. 
# E.g. the 'Indicator' column in the heart disease data, or the 'combined OD1 ... combined odX' data in the crimelab data
!pip install pymedtermino

In [ ]:
def find_data_relevant_for_tagging(df):
    